Gcloud setting needed. See https://cloud.google.com/sdk/docs/install-sdk

List clusters

In [1]:
! gcloud container clusters list

NAME: lab-cluster
LOCATION: europe-west4-a
MASTER_VERSION: 1.27.3-gke.100
MASTER_IP: 34.91.224.227
MACHINE_TYPE: e2-medium
NODE_VERSION: 1.27.3-gke.100
NUM_NODES: 3
STATUS: RUNNING


Connect google cloud to local kubectl CLI. Replace your cluster name with lab-cluster

In [3]:
! gcloud container clusters get-credentials lab-cluster

Fetching cluster endpoint and auth data.
kubeconfig entry generated for lab-cluster.


Enable the Filestore CSI

In [4]:
! gcloud container clusters update lab-cluster --update-addons=GcpFilestoreCsiDriver=ENABLED

Updating lab-cluster...
done.
Updated [https://container.googleapis.com/v1/projects/cryptic-dahlia-413216/zones/europe-west4-a/clusters/lab-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/europe-west4-a/lab-cluster?project=cryptic-dahlia-413216


RBAC settings

In [38]:
! kubectl apply -f security\admin-role.yaml
! kubectl apply -f security\reader-role.yaml
! kubectl apply -f security\editor-role.yaml
   
! kubectl apply -f security\admin-serviceaccount.yaml
! kubectl apply -f security\rolebinding.yaml
! kubectl apply -f security\user-rolebinding.yaml

role.rbac.authorization.k8s.io/admin-role unchanged
role.rbac.authorization.k8s.io/reader-role unchanged
role.rbac.authorization.k8s.io/editor-role unchanged
serviceaccount/admin-service-account unchanged
rolebinding.rbac.authorization.k8s.io/app-role-binding unchanged
rolebinding.rbac.authorization.k8s.io/user unchanged


Persistence storage claim

In [39]:
! kubectl apply -f gcloud\gcloud-pvc.yaml

persistentvolumeclaim/standard-pvc unchanged


Network settings

In [42]:
! kubectl apply -f db\mongodb-service.yaml
! kubectl apply -f backend\backend-service.yaml
! kubectl apply -f backend\service-loadbalancer.yaml
! kubectl apply -f app\app-loadbalancer.yaml

service/mongodb-service created
service/backend-service created
service/service-loadbalancer created
service/webapp-loadbalancer created


Get network IPs. If EXTERNAL-IP is pending, please wait for a while and re-execute

In [44]:
! kubectl get svc

NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
backend-service        ClusterIP      10.28.1.87     <none>          8000/TCP       69s
kubernetes             ClusterIP      10.28.0.1      <none>          443/TCP        28h
mongodb-service        ClusterIP      10.28.11.191   <none>          27017/TCP      70s
service-loadbalancer   LoadBalancer   10.28.13.11    34.141.152.76   80:31105/TCP   69s
webapp-loadbalancer    LoadBalancer   10.28.9.217    34.147.109.63   80:31788/TCP   69s


Put EXTERNAL-IP of service-loadbalancer to app\configmap-app.yaml
Put Cluster-IP of mongodb-service to backend\backend-config.yaml
```YAML
# app\configmap-app.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: server-address
data:
#  put the EXTERNAL-IP of serviceloadbalancer here
  SERVER_URL: "http://35.234.165.21"
```

```YAML
# backend\backend-service.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: backend-config
data:
# put the CLUSTER-IP of mongodb-service here
  MONGO_URL: "10.28.11.191"
```

Secret and configMap

In [45]:
! kubectl apply -f app\app-secret.yaml
! kubectl apply -f app\configmap-app.yaml
! kubectl apply -f backend\backend-secret.yaml
! kubectl apply -f backend\backend-config.yaml
! kubectl apply -f db\mongodb-secret.yaml
! kubectl apply -f db\mongodb-configmap.yaml

secret/webapp-registry-secret unchanged
configmap/server-address configured
secret/backend-secret unchanged
configmap/backend-config unchanged
secret/mongodb-secret unchanged
configmap/mongodb-config unchanged


Deploy database

In [46]:
! kubectl apply -f db\mongodb-deployment.yaml

deployment.apps/mongodb-deployment created


Set user/data to mongodb

In [47]:
! kubectl get pods

NAME                                  READY   STATUS              RESTARTS   AGE
mongodb-deployment-5b8bf7fbdf-tf2t4   0/1     ContainerCreating   0          2s


When the pod is running, go inside the pod and create database, set user and pass. The settings will be stored within persistence volume, so after manipulate pods, they remains.

```BASE
kubectl exec -it mongodb-deployment-5b8bf7fbdf-xp8zw -- bash
```

```BASH
mongosh
```

```javascript
use admin
```
```javascript
db.createUser({
  user: "admin",
  pwd: "pass",
  roles: ["userAdminAnyDatabase", "readWriteAnyDatabase"]
})
```
```javascript
db.auth("admin", "pass")
```

Deploy frontend and backend.

In [48]:
! kubectl apply -f app\app-deployment.yaml
! kubectl apply -f backend\backend-deployment.yaml

deployment.apps/webapp-deployment created
service/webapp-service created
deployment.apps/backend-deployment created


Show services, get the EXTERNAL-IP of webapp-loadbalancer, open it on web.

In [49]:
! kubectl get svc

NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)          AGE
backend-service        ClusterIP      10.28.1.87     <none>          8000/TCP         108s
kubernetes             ClusterIP      10.28.0.1      <none>          443/TCP          28h
mongodb-service        ClusterIP      10.28.11.191   <none>          27017/TCP        109s
service-loadbalancer   LoadBalancer   10.28.13.11    34.141.152.76   80:31105/TCP     108s
webapp-loadbalancer    LoadBalancer   10.28.9.217    34.147.109.63   80:31788/TCP     108s
webapp-service         NodePort       10.28.14.56    <none>          5000:30100/TCP   3s


List all resources we have

In [50]:
! kubectl get all

NAME                                      READY   STATUS    RESTARTS   AGE
pod/backend-deployment-c9f84c788-6rqn8    1/1     Running   0          28m
pod/mongodb-deployment-5b8bf7fbdf-tf2t4   1/1     Running   0          28m
pod/webapp-deployment-c54966686-xh7f5     1/1     Running   0          28m

NAME                           TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)          AGE
service/backend-service        ClusterIP      10.28.1.87     <none>          8000/TCP         30m
service/kubernetes             ClusterIP      10.28.0.1      <none>          443/TCP          28h
service/mongodb-service        ClusterIP      10.28.11.191   <none>          27017/TCP        30m
service/service-loadbalancer   LoadBalancer   10.28.13.11    34.141.152.76   80:31105/TCP     30m
service/webapp-loadbalancer    LoadBalancer   10.28.9.217    34.147.109.63   80:31788/TCP     30m
service/webapp-service         NodePort       10.28.14.56    <none>          5000:30100/TCP   28m

NAME        

Delete deployment

In [52]:
! kubectl delete deployment mongodb-deployment
! kubectl delete deployment backend-deployment
! kubectl delete deployment webapp-deployment

deployment.apps "mongodb-deployment" deleted


Error from server (NotFound): deployments.apps "backend-deployment" not found
Error from server (NotFound): deployments.apps "webapp-deployment" not found


Delete services

In [56]:
! kubectl delete svc mongodb-service
! kubectl delete svc service-loadbalancer
! kubectl delete svc webapp-loadbalancer
! kubectl delete svc webapp-service
! kubectl delete svc backend-service

Error from server (NotFound): services "mongodb-service" not found
Error from server (NotFound): services "service-loadbalancer" not found
Error from server (NotFound): services "webapp-loadbalancer" not found
Error from server (NotFound): services "webapp-service" not found


service "backend-service" deleted


Check resources

In [57]:
! kubectl get all

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.28.0.1    <none>        443/TCP   29h
